In [ ]:
import os
import torch

from torchvision.io import read_image
from torchvision.ops.boxes import masks_to_boxes
from torchvision import tv_tensors
from torchvision.transforms.v2 import functional as F
from google.colab import drive
from PIL import Image
import matplotlib.pyplot as plt
import sys
import pandas as pd
from collections import defaultdict
import json
import random
from torch.utils.data import Dataset
import numpy as np
import torchvision.models.detection as detection
from torch.utils.data import DataLoader
import torch.utils
import torch
from PIL import Image as read_image
from torchvision.io import read_image
from torchvision.transforms import v2
from transforms import RandomPhotometricDistort, RandomZoomOut, RandomIoUCrop, RandomHorizontalFlip,RandomVerticalFlip ,ToTensor, Compose
from torchvision.transforms.functional import convert_image_dtype

In [ ]:
drive.mount('/content/drive')
path_base = '/content/drive/MyDrive/Academico/TD8 personal '
sys.path.append(path_base)

Mounted at /content/drive


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:

train_file = path_base + '/classificationDownload (1)/train.csv'
test_file = path_base + '/classificationDownload (1)/test.csv'
classification_file = path_base + '/classificationDownload (1)/classifications.csv'

train_images = pd.read_csv(train_file)['image_filename'].tolist()
test_images = pd.read_csv(test_file)['image_filename'].tolist()

# Cargar la información completa desde classification.csv
classification_data = pd.read_csv(classification_file)

# Filtrar los datos de entrenamiento y prueba usando los nombres de las imágenes
train_data = classification_data[classification_data['image_filename'].isin(train_images)]
test_data = classification_data[classification_data['image_filename'].isin(test_images)]


def get_transform(train=True):
    transforms = []
    if train:
        transforms.append(RandomPhotometricDistort())
        transforms.append(RandomZoomOut(fill=list((207., 202., 205.))))
        transforms.append(RandomHorizontalFlip(p=0.5))
        transforms.append(RandomVerticalFlip(p=0.5))

    transforms.append(v2.ToDtype(torch.float, scale=True))
    return Compose(transforms)

def group_detections(data):
    grouped_data = {}
    for _, row in data.iterrows():
        image_name = row['image_filename']
        if image_name not in grouped_data:
            grouped_data[image_name] = {
                'image_filename': image_name,
                'image_id': row['image_id'],
                'image_doi': row['image_doi'],
                'classifications': []
            }
        grouped_data[image_name]['classifications'].append({
            'nucleus_x': row['nucleus_x'],
            'nucleus_y': row['nucleus_y'],
            'bethesda_system': row['bethesda_system'],
            'cell_id': row['cell_id']
        })
    return list(grouped_data.values())


train_data_grouped = group_detections(train_data)
test_data_grouped = group_detections(test_data)


train_counts = defaultdict(int)
test_counts = defaultdict(int)


for item in train_data_grouped:
    for classification in item['classifications']:
        train_counts[classification['bethesda_system']] += 1


for item in test_data_grouped:
    for classification in item['classifications']:
        test_counts[classification['bethesda_system']] += 1


class PAPDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

        self.label_map = {
            "SCC": 1,
            "HSIL": 2,
            "LSIL": 3,
            "ASC-H": 4,
            "ASC-US": 5,
            "Negative for intraepithelial lesion": 6
        }

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        img = read_image(path_base + '/Imagenes/' + item['image_filename'])

        image_doi = item['image_doi']
        image_id = item['image_id']
        boxes = []
        labels = []
        for classification in item['classifications']:
            nucleus_x = classification['nucleus_x']
            nucleus_y = classification['nucleus_y']
            x_min = nucleus_x - 50
            y_min = nucleus_y - 50
            x_max = nucleus_x + 50
            y_max = nucleus_y + 50
            boxes.append([x_min, y_min, x_max, y_max])
            labels.append(self.label_map[classification['bethesda_system']])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        target = {
            "boxes": boxes,
            "labels": labels,
            "area": area,
            "image_id": image_id
        }

        num_objs = boxes.shape[0]
        target["iscrowd"] = torch.zeros((num_objs,), dtype=torch.int64)

        if self.transform:
            img, target = self.transform(img, target)
        return img, target

train_dataset = PAPDataset(train_data_grouped, transform=get_transform(train=True))
test_dataset = PAPDataset(test_data_grouped, transform=get_transform(train=False))

print("Training Counts:", dict(train_counts))
print("Test Counts:", dict(test_counts))
print(f"Training Dataset Size: {len(train_dataset)}")
print(f"Test Dataset Size: {len(test_dataset)}")


Training Counts: {'SCC': 145, 'Negative for intraepithelial lesion': 6043, 'LSIL': 1221, 'HSIL': 1588, 'ASC-H': 808, 'ASC-US': 550}
Test Counts: {'ASC-H': 117, 'Negative for intraepithelial lesion': 736, 'ASC-US': 56, 'LSIL': 139, 'SCC': 16, 'HSIL': 115}
Training Dataset Size: 360
Test Dataset Size: 40


In [ ]:

from ssd import  ssd512_resnet50_Upgraded_version2
model = ssd512_resnet50_Upgraded_version2(load = False)
model = model.to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 179MB/s]


In [ ]:
from ssd import SSDClassificationHead, SSDRegressionHead
num_classes = 6+1  # Ejemplo con 6  clases + 1 clase background


in_channels = [2048, 512, 512, 256, 256, 256]
num_anchors = [4, 6, 6, 6, 4, 4]
model.head.classification_head = SSDClassificationHead(in_channels, num_anchors, num_classes)




In [ ]:

def collate_fn(batch):
    return tuple(zip(*batch))



batch_size = 16

train_sampler = torch.utils.data.RandomSampler(train_dataset)
test_sampler = torch.utils.data.SequentialSampler(test_dataset)

train_batch_sampler = train_batch_sampler = torch.utils.data.BatchSampler(
            train_sampler, batch_size, drop_last=True)


data_loader = torch.utils.data.DataLoader(
        train_dataset, batch_sampler=train_batch_sampler,
        collate_fn=collate_fn)


data_loader_test = torch.utils.data.DataLoader(
        test_dataset, batch_size=1,
        sampler=test_sampler,
        collate_fn=collate_fn)

In [ ]:
import math
from utils import MetricLogger, warmup_lr_scheduler, SmoothedValue
def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq):
    model.train()
    metric_logger = MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', SmoothedValue(window_size=1, fmt='{value:.6f}'))
    header = 'Epoch: [{}]'.format(epoch)

    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1. / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor)

    for images, targets in metric_logger.log_every(data_loader, print_freq, header):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) if not isinstance(v, int) else v for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        print(loss_value)

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            break

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

        metric_logger.update(lr=optimizer.param_groups[0]["lr"])

    return metric_logger



'\nMetricLogger te ayuda a monitorear métricas a lo largo del entrenamiento.\nWarmup Learning Rate Scheduler te permite empezar el entrenamiento de manera más suave ajustando gradualmente la tasa de aprendizaje durante las primeras iteraciones para evitar inestabilidad.\n'

In [ ]:
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py
!pip install -U pycocotools

--2024-11-19 17:52:59--  https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4063 (4.0K) [text/plain]
Saving to: ‘engine.py’

engine.py           100%[===================>]   3.97K  --.-KB/s    in 0s      

2024-11-19 17:52:59 (77.7 MB/s) - ‘engine.py’ saved [4063/4063]

--2024-11-19 17:52:59--  https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8388 (8.2K) [text/plain]
S

In [ ]:
from torchvision.datasets import VOCDetection
from torchvision.transforms import Compose, ToTensor, RandomHorizontalFlip, Normalize
from engine import evaluate
import time
import datetime


params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
        params, lr = 0.003, momentum=0.9, weight_decay = 0.0005)

lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[80,110], gamma=0.1)

model.to(device)
best_ap = 0.0

print("Start training")
start_time = time.time()
for epoch in range(50):
  train_one_epoch(model, optimizer, data_loader, device, epoch, 20)
  lr_scheduler.step()
  ap = evaluate(model, data_loader_test, device=device).coco_eval["bbox"].stats[0]
  if ap > best_ap:
    best_ap = ap
    best_model_weights = model.state_dict()

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))








Start training
45.61259460449219
Epoch: [0]  [ 0/22]  eta: 0:04:09  lr: 0.000146  time: 11.3590  data: 7.8418  max mem: 10243
46.07450485229492
43.192054748535156
36.659095764160156
31.2216854095459
22.64081382751465
22.624122619628906
16.06100082397461
13.106607437133789
12.44620132446289
11.814958572387695
10.642932891845703
12.137039184570312
9.993313789367676
11.239984512329102
10.757099151611328
8.809211730957031
8.956768035888672
11.865242004394531
10.446271896362305
8.101640701293945
Epoch: [0]  [20/22]  eta: 0:00:13  lr: 0.003000  time: 6.5237  data: 6.1692  max mem: 11049
8.039046287536621
Epoch: [0]  [21/22]  eta: 0:00:06  lr: 0.003000  time: 6.4942  data: 6.1408  max mem: 11049
Epoch: [0] Total time: 0:02:28 (6.7451 s / it)
creating index...
index created!
Test:  [ 0/40]  eta: 0:00:15  model_time: 0.2871 (0.2871)  evaluator_time: 0.0252 (0.0252)  time: 0.3791  data: 0.0629  max mem: 11049
Test:  [39/40]  eta: 0:00:00  model_time: 0.0169 (0.0240)  evaluator_time: 0.0196 (0.03

In [ ]:
print('El AP del mejor modelo siendo esre el que guardamos es:')
print(best_ap)
torch.save({
    'model_state_dict': best_model_weights,
}, path_base + "/pesos_modelos/SSD512_Resnet_upgraded_version2.pth" )

El AP del mejor modelo siendo esre el que guardamos es:
0.24600875456125984


In [ ]:

start_time = time.time()


for epoch in range(50, 100):
    train_one_epoch(model, optimizer, data_loader, device, epoch, 20)
    lr_scheduler.step()
    evaluate(model, data_loader_test, device=device)

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))



2.590792655944824
Epoch: [50]  [ 0/22]  eta: 0:00:56  lr: 0.003000  time: 2.5469  data: 2.1797  max mem: 11679
2.8015456199645996
2.6088457107543945
2.069963216781616
2.527776002883911
2.2109146118164062
2.5711746215820312
2.4822709560394287
2.655457019805908
2.530569314956665
2.5970680713653564
3.0356321334838867
2.5307250022888184
1.9947848320007324
2.9792897701263428
2.887033462524414
2.3377866744995117
2.0282909870147705
2.4052367210388184
2.876437187194824
2.5043325424194336
Epoch: [50]  [20/22]  eta: 0:00:04  lr: 0.003000  time: 2.2248  data: 1.8894  max mem: 11679
2.658013343811035
Epoch: [50]  [21/22]  eta: 0:00:02  lr: 0.003000  time: 2.2026  data: 1.8599  max mem: 11679
Epoch: [50] Total time: 0:00:49 (2.2485 s / it)
creating index...
index created!
Test:  [ 0/40]  eta: 0:00:04  model_time: 0.0179 (0.0179)  evaluator_time: 0.0235 (0.0235)  time: 0.1097  data: 0.0643  max mem: 11679
Test:  [39/40]  eta: 0:00:00  model_time: 0.0172 (0.0177)  evaluator_time: 0.0201 (0.0321)  tim

In [ ]:
print('El AP del mejor modelo siendo esre el que guardamos es:')
print(best_ap)
torch.save({
    'model_state_dict': best_model_weights,
}, path_base + "/pesos_modelos/SSD512_Resnet_upgraded_version2.pth" )

El AP del mejor modelo siendo esre el que guardamos es:
0.24600875456125984


In [ ]:
start_time = time.time()


for epoch in range(100, 120):
    train_one_epoch(model, optimizer, data_loader, device, epoch, 20)
    lr_scheduler.step()
    ap = evaluate(model, data_loader_test, device=device).coco_eval["bbox"].stats[0]
    if ap > best_ap:
      best_ap = ap
      best_model_weights = model.state_dict()

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))

1.9309526681900024
Epoch: [100]  [ 0/22]  eta: 0:00:45  lr: 0.000300  time: 2.0756  data: 1.8128  max mem: 11679
1.8839389085769653
2.502800941467285
2.7455475330352783
2.2066264152526855
2.203341007232666
1.6372121572494507
2.103743553161621
2.491570472717285
1.9445815086364746
2.5342812538146973
1.7187913656234741
2.0523102283477783
2.2052724361419678
1.7929986715316772
2.069805860519409
2.0940051078796387
2.351483106613159
2.1289455890655518
2.0817461013793945
2.669309616088867
Epoch: [100]  [20/22]  eta: 0:00:04  lr: 0.000300  time: 2.1914  data: 1.8426  max mem: 11679
1.8235177993774414
Epoch: [100]  [21/22]  eta: 0:00:02  lr: 0.000300  time: 2.1645  data: 1.8173  max mem: 11679
Epoch: [100] Total time: 0:00:47 (2.1786 s / it)
creating index...
index created!
Test:  [ 0/40]  eta: 0:00:04  model_time: 0.0180 (0.0180)  evaluator_time: 0.0155 (0.0155)  time: 0.1007  data: 0.0631  max mem: 11679
Test:  [39/40]  eta: 0:00:00  model_time: 0.0177 (0.0177)  evaluator_time: 0.0225 (0.0289)

In [ ]:
print('El AP del mejor modelo siendo esre el que guardamos es:')
print(best_ap)
torch.save({
    'model_state_dict': best_model_weights,
}, path_base + "/pesos_modelos/SSD512_Resnet_upgraded_version2.pth" )

El AP del mejor modelo siendo esre el que guardamos es:
0.29992954309173875


In [ ]:
start_time = time.time()


for epoch in range(120, 150):
    train_one_epoch(model, optimizer, data_loader, device, epoch, 20)
    lr_scheduler.step()
    ap = evaluate(model, data_loader_test, device=device).coco_eval["bbox"].stats[0]
    if ap > best_ap:
      best_ap = ap
      best_model_weights = model.state_dict()

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))

2.003321409225464
Epoch: [120]  [ 0/22]  eta: 0:00:43  lr: 0.000030  time: 1.9988  data: 1.6727  max mem: 11755
2.0008416175842285
1.9945337772369385
2.0427253246307373
2.0626914501190186
2.114922046661377
2.135418653488159
2.8965415954589844
1.9092590808868408
2.493338108062744
2.4590611457824707
2.173083782196045
2.2554514408111572
2.161667585372925
1.7787177562713623
2.1824560165405273
2.0592329502105713
2.782029151916504
2.1948370933532715
1.745648741722107
2.0116047859191895
Epoch: [120]  [20/22]  eta: 0:00:04  lr: 0.000030  time: 2.2057  data: 1.8536  max mem: 11755
1.9417980909347534
Epoch: [120]  [21/22]  eta: 0:00:02  lr: 0.000030  time: 2.1641  data: 1.8207  max mem: 11755
Epoch: [120] Total time: 0:00:48 (2.1880 s / it)
creating index...
index created!
Test:  [ 0/40]  eta: 0:00:04  model_time: 0.0180 (0.0180)  evaluator_time: 0.0155 (0.0155)  time: 0.1014  data: 0.0639  max mem: 11755
Test:  [39/40]  eta: 0:00:00  model_time: 0.0173 (0.0176)  evaluator_time: 0.0211 (0.0289) 

In [ ]:
print('El AP del mejor modelo siendo esre el que guardamos es:')
print(best_ap)
torch.save({
    'model_state_dict': best_model_weights,
}, path_base + "/pesos_modelos/SSD512_Resnet_upgraded_version2.pth"  )

El AP del mejor modelo siendo esre el que guardamos es:
0.29992954309173875
